In [1]:
import requests
import pandas as pd

url = 'https://github.com/nytimes/covid-19-data/blob/master/us-states.csv'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1]
print(df)
df.to_csv('my data.csv')

       Unnamed: 0        date          state  fips   cases  deaths
0             NaN  2020-01-21     Washington    53       1       0
1             NaN  2020-01-22     Washington    53       1       0
2             NaN  2020-01-23     Washington    53       1       0
3             NaN  2020-01-24       Illinois    17       1       0
4             NaN  2020-01-24     Washington    53       1       0
...           ...         ...            ...   ...     ...     ...
10899         NaN  2020-09-16       Virginia    51  136359    2884
10900         NaN  2020-09-16     Washington    53   84714    2113
10901         NaN  2020-09-16  West Virginia    54   13196     290
10902         NaN  2020-09-16      Wisconsin    55   98501    1238
10903         NaN  2020-09-16        Wyoming    56    4566      46

[10904 rows x 6 columns]


In [2]:
df=pd.read_csv("my data.csv").drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)
df.tail()

,date,state,fips,cases,deaths
10899,2020-09-16,Virginia,51,136359,2884
10900,2020-09-16,Washington,53,84714,2113
10901,2020-09-16,West Virginia,54,13196,290
10902,2020-09-16,Wisconsin,55,98501,1238
10903,2020-09-16,Wyoming,56,4566,46


In [3]:
import numpy as  np
np.unique(df[["state"]])

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Northern Mariana Islands',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virgin Islands', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [4]:
ca=df[df["state"]=="California"]
ca

,date,state,fips,cases,deaths
5,2020-01-25,California,6,1,0
9,2020-01-26,California,6,2,0
13,2020-01-27,California,6,2,0
17,2020-01-28,California,6,2,0
21,2020-01-29,California,6,2,0
...,...,...,...,...,...
10633,2020-09-12,California,6,760581,14333
10688,2020-09-13,California,6,763389,14386
10743,2020-09-14,California,6,767062,14458
10798,2020-09-15,California,6,769917,14606


In [5]:
from matplotlib import pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import math

In [28]:
X = ca[["cases"]].values
size = int(len(X) * 0.95)
train, test = X[0:size], X[size:len(X)]

In [29]:
history = [x for x in train]
predictions = list()
for t in range(len(test)):
	model = ARIMA(history, order=(2,1,2))
	model_fit = model.fit(disp=-1)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
mape = np.mean(np.abs(predictions -test )/np.abs(test))
print('MAPE: '+str(mape))
error = mean_squared_error(test, predictions)
RMSE=math.sqrt(error)
print("RMSE: ", RMSE)

MAPE: 0.000914175097022744
RMSE:  772.3161171382758


In [30]:
fc, se, conf=model_fit.forecast(len(test)+7,alpha=0.01)

In [31]:
fc=pd.DataFrame(fc).rename(columns={0:"forecast"})
fc.tail(7)

,forecast
12,810041.659164
13,813100.213627
14,816153.408930
15,819201.297469
16,822243.931122
17,825281.361266
18,828313.638770


In [32]:
se=pd.DataFrame(conf).rename(columns={0:"lower bound",1:"upper bound"})
se.tail(7)

,lower bound,upper bound
12,777078.916328,843004.402000
13,776904.249129,849296.178124
14,776642.673854,855664.144006
15,776298.150344,862104.444593
16,775874.359527,868613.502718
17,775374.732714,875187.989818
18,774802.477325,881824.800215


In [33]:
date=ca[['date']]
train_date, test_date = date[0:size], date[size:len(X)]
test_date.head()

,date
10248,2020-09-05
10303,2020-09-06
10358,2020-09-07
10413,2020-09-08
10468,2020-09-09


In [34]:
import datetime
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)


days=daterange(datetime.datetime(2020, 9,5),
               datetime.datetime(2020, 9, 23))
days=pd.DataFrame(days).rename(columns={0:"Date"})
days.tail(7)

,Date
12,2020-09-17
13,2020-09-18
14,2020-09-19
15,2020-09-20
16,2020-09-21
17,2020-09-22
18,2020-09-23


In [40]:
ca_cases=pd.concat([days,fc,se],axis=1)
ca_cases.tail(7)

,Date,forecast,lower bound,upper bound
12,2020-09-17,810041.659164,777078.916328,843004.402000
13,2020-09-18,813100.213627,776904.249129,849296.178124
14,2020-09-19,816153.408930,776642.673854,855664.144006
15,2020-09-20,819201.297469,776298.150344,862104.444593
16,2020-09-21,822243.931122,775874.359527,868613.502718
17,2020-09-22,825281.361266,775374.732714,875187.989818
18,2020-09-23,828313.638770,774802.477325,881824.800215


In [41]:
difference = ca_cases.diff(axis=0);

print("Difference between rows(Period=1):");

difference=pd.DataFrame(difference).drop(["Date"],axis=1).rename(columns={"forecast":"Daily change"})
difference=pd.concat([days,difference],axis=1)
difference.tail(7)

Difference between rows(Period=1):


,Date,Daily change,lower bound,upper bound
12,2020-09-17,3063.966532,-83.487346,6211.420410
13,2020-09-18,3058.554462,-174.667199,6291.776123
14,2020-09-19,3053.195304,-261.575275,6367.965883
15,2020-09-20,3047.888538,-344.523510,6440.300587
16,2020-09-21,3042.633654,-423.790818,6509.058125
17,2020-09-22,3037.430143,-499.626813,6574.487099
18,2020-09-23,3032.277504,-572.255389,6636.810397


The model seems to overestimate the total cases but underestimate the daily changes. The lower bound is closer to the reported numbers. A LSTM neural network model might do a better job. Still, the rolling ARIMA model forecast that daily new cases will continue to decrease in the days to come. California is gradually flattening the curve. 

However, that does not Californians could relax, get rid of face masks, have huge gathering, etc. As the situation stands, new daily cases are still forecast to be above 3000. The projections show that what we have been doing are working and we should continue what we have been doing regarding the coronavirus pandemic, till it is controlled or even eliminated. 